In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# ----- Physical and numerical parameters -----
hbar = 1.054571817e-34 
m = 9.10938356e-31      
L = 1e-8                
N = 1000               
a = L / N              
h = 1e-18               


x = np.linspace(a, L - a, N - 1)


x0 = L / 2
sigma = 1e-10
kappa = 5e10
psi0 = np.exp(-((x - x0)**2) / (2 * sigma**2)) * np.exp(1j * kappa * x)

# ----- Crank–Nicolson coefficients -----
alpha = (1j * hbar * h) / (4 * m * a**2)
a1 = 1 + 2 * alpha
a2 = -alpha
b1 = 1 - 2 * alpha
b2 = alpha


size = N - 1
A = np.zeros((size, size), dtype=complex)
B = np.zeros((size, size), dtype=complex)

for i in range(size):
    A[i, i] = a1
    B[i, i] = b1
    if i > 0:
        A[i, i-1] = a2
        B[i, i-1] = b2
    if i < size - 1:
        A[i, i+1] = a2
        B[i, i+1] = b2

# ----- One Crank–Nicolson step -----
def crank_nicolson_step(psi):
    v = B @ psi
    psi_next = np.linalg.solve(A, v)
    return psi_next

def normalize(psi, dx=a):
    norm = np.sqrt(np.sum(np.abs(psi)**2) * dx)
    return psi / norm if norm != 0 else psi


psi = psi0.copy()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

line1, = ax1.plot(x, np.real(psi), lw=2)
line2, = ax2.plot(x, np.abs(psi)**2, lw=2)

ax1.set_xlim(0, L)
ax2.set_xlim(0, L)

ax1.set_xlabel("x (m)")
ax1.set_ylabel("Re(ψ)")
ax1.set_title("Real part of ψ")

ax2.set_xlabel("x (m)")
ax2.set_ylabel("|ψ|²")
ax2.set_title("Probability density")


def update(frame):
    global psi
    psi = crank_nicolson_step(psi)
    psi = normalize(psi)

    # Update real part
    line1.set_ydata(np.real(psi))
    max_real = np.max(np.abs(np.real(psi)))
    ax1.set_ylim(-1.1*max_real, 1.1*max_real)

    # Update probability density
    line2.set_ydata(np.abs(psi)**2)
    max_prob = np.max(np.abs(psi)**2)
    ax2.set_ylim(0, 1.1*max_prob)

    return line1, line2

anim = animation.FuncAnimation(fig, update, frames=300, interval=30, blit=True)

anim.save("wavefunction_evolution.gif", writer="pillow", fps=30, dpi=100)

plt.close(fig) 
